# Midterm — iNaturalist
### Daniel Phillips, Chris Howard, Phillip Johnson, Jacob Smith, Michael Reid

In [1]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display

def get_taxa_id(species_name):
    '''
    This function returns the taxon_id when given the species name.
    
    Parameters:
    species_name: a string object representing a species name, e.g. "Danaus plexippus"
    
    Returns: ids, a list object containing integer id's for the species
    '''
    
    base_url = "http://api.inaturalist.org/v1/taxa/autocomplete?q="

    
    request = requests.get(base_url + "%20".join(species_name.split()))
    data = request.json()

    ids = []
    for i in data['results']:
        ids.append(i['id'])

    return ids


def get_observation(id_no, month, year):
    '''
    This function returns observation data when given taxon_id, month, and year.
    
    Parameters: 
    id_no: an integer representing species taxon_id
    month: an integer (1-12) representing the month of interest
    year: an integer representing year of interest
    
    Returns: observational data for taxon_id for specified month and year.
    '''
    #Url builder, for the request
    base_url = "http://api.inaturalist.org/v1/observations?"
    end_url = "&order=desc&order_by=created_at"
    url = base_url + 'taxon_id=' + str(id_no) + '&month=' + str(month) + '&year=' + str(year) + end_url

    request = requests.get(url)
    data = request.json()    

    return data
    
        
def get_count_one_month(id_no_lst, month, year):
    '''
    This function counts the number of observations of a taxon_id, for each month of a given year.
    
    Parameters:
    id_no_lst: a Python list object containing IDs 
    month: an integer object (1-12) representing the month you want the count for
    year: an integer object for the year of interest
    
    Returns: count, an integer of how many observations are given for some id, for some month of a given year.
    '''
    count = 0
    for i in id_no_lst:
        count += int(get_observation(i, month, year)['total_results']) #total_results key associates w/ ea. set of obs data
    return count
    
    

species = [
    'Danaus plexippus',
    'Hyles lineata',
    'Zerene cesonia',
    'Papilio multicaudata',
    'Agraulis vanillae',
    'Papilio cresphontes',
    'Strymon melinus',
    'Vanessa cardui',
    'Hylephila phyleus',
    'Danaus gilippus'
]

months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

def main():
    print('running')
    species_to_id = {}
    frames = []
    
    
    #Get a dictionary of the taxa -> lst(ids)
    for i in species:
        species_to_id[i] = get_taxa_id(i)
    
    #Map integers 1-12 to 'January' through 'December'
    month_map = dict(zip(range(1,13), months))
    
    species_dict_out = {}
    year = 2016
    
    print(species_to_id)
    
    #Create a dictionary for each species
    for spec in species_to_id:
        species_dict_out[spec] = {}
        
        #Map each species' months to their corresponding count of species observations for that month (and year)
        for mon in month_map:
            species_dict_out[spec][month_map[mon]] = get_count_one_month(species_to_id[spec], mon, year)
        
        print(species_dict_out[spec])
        
        #Creates list of observed IDs for each species, e.g. {'Danaus plexippus': [48662, 235550]}
        frames.append(species_dict_out[spec])
            
        
    #Makes the JSON Species->ID_List structures 
    result = json_normalize(frames)
    display(result)
    
    
%time main()
    

running
{'Hyles lineata': [49348, 293806], 'Hylephila phyleus': [50340, 312192], 'Strymon melinus': [50931, 346149, 238732, 238733, 542113, 238734, 238735], 'Danaus gilippus': [51743, 51748, 50063, 51744, 51746, 51747, 51749], 'Papilio multicaudata': [68263, 237647, 148765, 460141, 545972], 'Agraulis vanillae': [49150, 312743, 461059, 416323, 208039, 234216], 'Papilio cresphontes': [50072], 'Zerene cesonia': [129362, 239037, 532836], 'Danaus plexippus': [48662, 235550], 'Vanessa cardui': [48548]}
{'August': 70, 'December': 1, 'June': 26, 'March': 73, 'July': 67, 'February': 7, 'May': 43, 'April': 59, 'September': 50, 'October': 16, 'January': 1, 'November': 5}
{'August': 224, 'December': 17, 'June': 116, 'March': 43, 'July': 148, 'February': 12, 'May': 65, 'April': 72, 'September': 292, 'October': 280, 'January': 6, 'November': 131}
{'August': 163, 'December': 2, 'June': 100, 'March': 16, 'July': 123, 'February': 29, 'May': 99, 'April': 128, 'September': 220, 'October': 242, 'January':

,April,August,December,February,January,July,June,March,May,November,October,September
0,59,70,1,7,1,67,26,73,43,5,16,50
1,72,224,17,12,6,148,116,43,65,131,280,292
2,128,163,2,29,4,123,100,16,99,29,242,220
3,23,131,86,12,19,93,58,16,37,251,373,212
4,15,41,4,2,0,33,40,7,16,7,24,37
5,86,245,79,33,31,207,104,44,92,395,707,352
6,9,104,3,5,1,54,40,10,19,12,76,73
7,7,15,21,1,3,9,5,1,5,110,91,57
8,306,745,288,180,105,418,226,283,219,618,1269,988
9,61,88,38,24,12,123,98,32,47,164,105,76


Wall time: 4min 5s


In [2]:
taxon_dict={}
def get_organized(file):
	file=open(file, 'r')
	for line in file:
		line=line.split()
		taxon_dict[line[0]]=' '.join(line[1:])
		print(taxon_dict[line[0]])
	return taxon_dict
		
hello = get_organized('taxon_list.txt')
print(taxon_dict)

scientificName
Polites origenes
Epargyreus clarus
Poanes zabulon
Euphyes dion
Pyrgus scriptura
Thorybes pylades
Erynnis juvenalis
Polites mystic
Ancyloxypha numitor
Poanes hobomok
Thymelicus lineola
Erynnis baptisiae
Polites themistocles
Eantis tamenund
Thorybes drusius
Antigonus emorsa
Chioides albofasciatus
Urbanus proteus
Poanes viator
Urbanus dorantes
Pyrgus centaureae
Pyrgus albescens
Wallengrenia egeremet
Euphyes vestris
Panoquina ocola
Hylephila phyleus
Pyrgus communis
Achlyodes pallida
Thorybes confusis
Lerema accius
Pyrgus oileus
Hesperia columbia
Erynnis horatius
Amblyscirtes celia
Hesperia colorado
Thorybes bathyllus
Autochton pseudocellus
Heliopetes macaira
Heliopetes laviana
Poanes melane
Polites peckius
Gesta invisus
Amblyscirtes fimbriata
Erynnis zarucco
Lerodea eufala
Phocides pigmalion
Pholisora catullus
Ochlodes agricola
Atalopedes campestris
Wallengrenia otho
Polites sabuleti
Erynnis tristis
Erynnis funeralis
Erynnis icelus
Amblyscirtes hegon
Carterocephalus palaemon

## Running the R Scripts through Bash

In [3]:
%%bash
ids=("20000" "50931")
for id in "${ids[@]}";
    do Rscript --vanilla run-sdm.R data/inaturalist/$id-iNaturalist.txt $id output/
done

Fatal error: cannot open file 'run-sdm.R': No such file or directory
Fatal error: cannot open file 'run-sdm.R': No such file or directory


### a note about this bash script

The above script will work if the notebook is sitting in the same directory as the R scripts (like run-sdm.R). It iterates through each taxon ID in the ids array and runs the model on its corresponding text file (the CSV returned from Jeff's get-observations script). When we have our CSVs written, we can use this on them. We just have to think about how we want to structure the directory. In any case, this produces the grid files and the raster, and then outputs this to Jupyter (for these two taxon id's):

null device 
          1 
class       : RasterLayer 
dimensions  : 2664, 7872, 20971008  (nrow, ncol, ncell)
resolution  : 0.04166667, 0.04166667  (x, y)
extent      : -151, 177, -46, 65  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 
data source : /mnt/c/Users/Chris/Desktop/ebutterfly-sdm/scripts/output/20000-prediction.grd 
names       : layer 
values      : 0, 0.6247379  (min, max)

class       : RasterLayer 
dimensions  : 2664, 7872, 20971008  (nrow, ncol, ncell)
resolution  : 0.04166667, 0.04166667  (x, y)
extent      : -151, 177, -46, 65  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 
data source : /mnt/c/Users/Chris/Desktop/ebutterfly-sdm/scripts/output/20000-prediction-threshold.grd 
names       : layer 
values      : 0, 1  (min, max)

Finished with file writing.
null device 
          1 
class       : RasterLayer 
dimensions  : 1272, 1392, 1770624  (nrow, ncol, ncell)
resolution  : 0.04166667, 0.04166667  (x, y)
extent      : -125, -67, -1, 52  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 
data source : /mnt/c/Users/Chris/Desktop/ebutterfly-sdm/scripts/output/50931-prediction.grd 
names       : layer 
values      : 0, 0.5571302  (min, max)

class       : RasterLayer 
dimensions  : 1272, 1392, 1770624  (nrow, ncol, ncell)
resolution  : 0.04166667, 0.04166667  (x, y)
extent      : -125, -67, -1, 52  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 
data source : /mnt/c/Users/Chris/Desktop/ebutterfly-sdm/scripts/output/50931-prediction-threshold.grd 
names       : layer 
values      : 0, 1  (min, max)

Finished with file writing.